In [78]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tempfile
import flategy as F
import subprocess
import multiprocessing
import os
import time
import cairo
import math
import io
import enum
import IPython.display
from IPython.display import display, SVG, Video

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [79]:
class UnitType(enum.Enum):
    BLOCK = 1
    OBJECTIVE = 2
    HEADQUARTERS = 3
    TURRET = 4
    MELEE = 5
    RANGED = 6
    FAST = 7

class InstructionType(enum.Enum):
    STOP = 1
    DEFEND = 2
    MOVE = 3
    ATTACK = 4

In [80]:
def get_collisions(base_position, position, base_radius, radius):
    return ((position[:, np.newaxis] - base_position[np.newaxis]) ** 2).sum(-1) < (radius[:, np.newaxis] + base_radius[np.newaxis]) ** 2

def move(state, delta):
    """Move objects, with basic collision avoidance with backtracking."""
    new_position = np.clip(state.position + delta,
                           state.world_shape[0] + state.radius[..., np.newaxis],
                           state.world_shape[1] - state.radius[..., np.newaxis])
    not_self = ~np.eye(len(state.position), dtype=np.bool)
    old_collisions = get_collisions(new_position, state.position, state.radius, state.radius) * not_self
    new_collisions = get_collisions(new_position, new_position, state.radius, state.radius) * not_self
    position = np.where(new_collisions.sum(-1) | old_collisions.sum(-1) | old_collisions.sum(-2), state.position, new_position)
    return state.replace(position=position)

In [81]:
state = F.State(
    position=np.array([[0.1, 0.5], [1, 0.5]]),
    radius=np.array([0.05, 0.2]),
    world_shape=np.array([[0, 0], [2, 1]]),
)

delta = 0.002 * np.array([[1, 0], [0, 0]])

states = []
for n in range(501):
    state = move(state, delta)
    if n % 10 == 0:
        states.append(state)
F.State.video(states, 'eg_new.mp4', 0.1, 256)